In [1]:
!pip install datasets
!pip install torch
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, GPT2Tokenizer, Trainer, TrainingArguments, AutoModelForCausalLM, GPT2LMHeadModel
import datasets
import pickle
import pandas as pd
import math
import torch

In [4]:
with open('/content/drive/MyDrive/UCLA/CS263/Proj/train_data.pkl', 'rb') as file:
    train_data_np = pickle.load(file)
with open('/content/drive/MyDrive/UCLA/CS263/Proj/test_data.pkl', 'rb') as file:
    test_data_np = pickle.load(file)

In [5]:
def parse_data(line):
    line = line.replace("[mask]", "<mask>")
    input, output = line.split("[sep]")

    ret = input + "<fill> " + output
    ret = ret.replace("  ", " ")
    ret = ret.replace(" ,", ",")
    ret = ret.strip()

    return ret

model_checkpoint = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer.pad_token = tokenizer.eos_token

print(train_data_np[0])
print(test_data_np[0])

train_data = [parse_data(s) for s in train_data_np]
test_data = [parse_data(s) for s in test_data_np]

print(train_data[0])
print(test_data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Major League [mask] will be speaking at St. Norbert College next month. [sep] Major League Baseball Commissioner Bud Selig will be speaking at St. Norbert College next month.
The NBA officially released its preseason schedule today, complete [mask] sent out to the media. [sep] The NBA officially released its preseason schedule today, complete with a press release sent out to the media.
Major League <mask> will be speaking at St. Norbert College next month. <fill> Major League Baseball Commissioner Bud Selig will be speaking at St. Norbert College next month.
The NBA officially released its preseason schedule today, complete <mask> sent out to the media. <fill> The NBA officially released its preseason schedule today, complete with a press release sent out to the media.


In [6]:
df_train = pd.DataFrame({'t': train_data, 'l': train_data})
df_test = pd.DataFrame({'t': test_data, 'l': test_data})

train_dataset = datasets.Dataset.from_pandas(df_train)
test_dataset = datasets.Dataset.from_pandas(df_test)

dataset = datasets.DatasetDict({"train": train_dataset, "test": test_dataset})

def tokenize_l(examples):
    return tokenizer(examples["l"], truncation=False, padding='max_length', max_length=64)

def tokenize_t(examples):
    return tokenizer(examples["t"], truncation=False, padding='max_length', max_length=64)

lm_datasets = dataset.map(tokenize_l, batched=True, remove_columns=["l"])
lm_datasets = lm_datasets.rename_column("input_ids", "labels")
lm_datasets = lm_datasets.remove_columns("attention_mask")
lm_datasets = lm_datasets.map(tokenize_t, batched=True, remove_columns=["t"])

Map:   0%|          | 0/60073 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/60073 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [7]:
lm_datasets = lm_datasets.filter(lambda example: len(example['input_ids']) <= 64)

print(lm_datasets)

Filter:   0%|          | 0/60073 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 59180
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 20
    })
})


In [8]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"/content/drive/MyDrive/UCLA/CS263/Proj/{model_name}-finetuned-infill",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

trainer.train()

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.369300,1.068166


Perplexity: 2.91


In [12]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/UCLA/CS263/Proj/gpt2-finetuned-infill/checkpoint-7000")
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)

In [13]:
device = torch.device("cuda")

# Generate a few words from an input sentence
for text in test_data:
    input_sentence = text.split(" <fill>")[0]
    print(text)

    with torch.no_grad():
        input_ids = tokenizer(input_sentence, return_tensors="pt")
        if len(input_ids) > 48:
            break
        assert torch.isfinite(input_ids['input_ids']).all(), "Input tensor contains NaN or Inf values before moving to device"
        # Move to the appropriate device
        #input_ids = input_ids.to(device)
        # Check for NaNs or Infs after moving to the device
        assert torch.isfinite(input_ids['input_ids']).all(), "Input tensor contains NaN or Inf values after moving to device"

        output_ids = model.generate(**input_ids, max_length=64, num_return_sequences=1, do_sample=True, pad_token_id=50256)
        output_words = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        print("Output: " + output_words.split("<fill> ")[-1])

The NBA officially released its preseason schedule today, complete <mask> sent out to the media. <fill> The NBA officially released its preseason schedule today, complete with a press release sent out to the media.
Output: The NBA officially released its preseason schedule today, complete with the Celtics sent out to the media.
French government officials and citizens have embraced Burmese <mask> Suu Kyi, who is within days of ... <fill> French government officials and citizens have embraced Burmese democracy advocate Aung San Suu Kyi, who is within days of ...
Output: French government officials and citizens have embraced Burmese refugees on Monday, giving Suu Kyi...
India and Sri Lanka today agreed not to resort to violence against fishermen straying <mask> maritime boundary. <fill> India and Sri Lanka today agreed not to resort to violence against fishermen straying into each-other's maritime boundary.
Output: India and Sri Lanka today agreed not to resort to violence against fisher

In [15]:
save_directory = "/content/drive/MyDrive/UCLA/CS263/Proj/insert_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/UCLA/CS263/Proj/insert_model/tokenizer_config.json',
 '/content/drive/MyDrive/UCLA/CS263/Proj/insert_model/special_tokens_map.json',
 '/content/drive/MyDrive/UCLA/CS263/Proj/insert_model/vocab.json',
 '/content/drive/MyDrive/UCLA/CS263/Proj/insert_model/merges.txt',
 '/content/drive/MyDrive/UCLA/CS263/Proj/insert_model/added_tokens.json')